# _**Baselines and Content-Based Recommender**_ #

In [5]:
import os
import math
import json
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random
import warnings
warnings.filterwarnings('ignore')

In [6]:
movies = pd.read_csv("../data/final data/movies_metadata.csv")
ratings = pd.read_csv("../data/final data/ratings_small.csv")
links = pd.read_csv("../data/final data/links_small.csv")
credits = pd.read_csv("../data/final data/credits.csv")
keywords = pd.read_csv("../data/final data/keywords.csv")